# Реализация линейной регрессии в TensorFlow

In [1]:
import numpy as np
import tensorflow as tf

print(tf.__version__)
print(tf.executing_eagerly())

2.2.0
True


## Генерируем данные для задачи регрессии

In [ ]:
NUM_FEATURES = 1
NUM_SAMPLES = 995

In [ ]:
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split

X, y = make_regression(n_samples = NUM_SAMPLES,
                       n_features = NUM_FEATURES,
                       n_informative = NUM_FEATURES,
                       bias=2.5,
                       noise = 10,
                       random_state = 12345)

y = y.reshape(-1, 1)

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

plt.figure(figsize=(10, 8))
plt.plot(X[:, 0], y, 'o')
plt.xlabel('x')
plt.ylabel('y')

plt.show()

## Функции для обучения модели

In [ ]:
@tf.custom_gradient
def linear(x, w, b):
    x = tf.convert_to_tensor(x)
    result = tf.matmul(x, w) + b
    def grad(dy):
        # возвращаем значения градиента для
        # каждого из аргументов функции
        dx = dy * w
        dw = tf.reduce_sum(dy * x, axis=0, keepdims=True)
        db = tf.reduce_sum(dy)
        return dx, dw, db
    return result, grad

def mse_loss(y, prediction):
    y = tf.convert_to_tensor(y)
    loss = (y - prediction) ** 2
    return tf.reduce_mean(loss)

## Обучение модели

In [ ]:
# создаем переменные для хранения весов модели
np.random.seed(12345)

w = tf.Variable(
    np.random.normal(size=(NUM_FEATURES, 1)),
    dtype=tf.double, name='w')

b = tf.Variable(0.0, dtype=tf.double, name='b')


# разделяем выборку на обучающую и на тестовую группы
(X_train, X_test,
 y_train, y_test) = train_test_split(X, y, 
                                     test_size=0.33,
                                     random_state=12345)

test_loss_results = []

lr = 2.5e-3  # скорость обучения

for i in range(500):
    with tf.GradientTape() as g:
        # вычисляем значение функции потерь для обучающей выборки
        y_pred = linear(X_train, w, b)
        loss = mse_loss(y_train, y_pred)

    # вычисляем значение градиента для параметров модели
    dw, db = g.gradient(loss, [w, b])

    # обновляем параметры модели с учетом значений градиентов
    w.assign_sub(lr * dw)
    b.assign_sub(lr * db)

    if i % 50 == 0:
        # логируем метрики на тестовой выборке
        y_pred = linear(X_test, w, b)
        test_loss_results.append(mse_loss(y_test, y_pred).numpy())
        print('[%03d] mse=%.3f' % (i, test_loss_results[-1]))

## Визуализируем кривую обучения

In [ ]:
plt.figure(figsize=(10, 4))
plt.plot(test_loss_results)
plt.title('Metrics')
plt.xlabel('Epoch')
plt.ylabel('MSE')
plt.show()

## Визуализируем разделяющую гиперплоскость 

In [ ]:
y_pred = linear(X_test, w, b).numpy()

order = np.argsort(X_test[:, 0])

plt.figure(figsize=(10, 8))
plt.title('Line: w=%.3f b=%.3f' % (w.numpy(), b.numpy()))

plt.plot(X_test, y_test, 'ob',
         X_test[order], y_pred[order], '-r')

plt.xlabel('x')
plt.ylabel('y')

plt.show()